In [172]:
import re

SLOPE_REGEX = re.compile(r">|v|<")


def get_end_position(input_string: str) -> complex:
    game_map = input_string.splitlines()
    for x in range(len(game_map[0])):
        if game_map[len(game_map) - 1][x] == ".":
            return complex(len(game_map) - 1, x)


def iterate(
    input_string: str,
    start_position: complex,
    end_position: complex,
    visited,
):
    steps = 0
    position = start_position
    game_map = input_string.splitlines()

    while position != end_position:
        visited.add(position)
        next_positions = map(lambda move: position + move, [1j, -1j, 1, -1])
        next_positions = list(
            filter(
                lambda pos: pos not in visited
                and pos.imag >= 0
                and pos.real >= 0
                and pos.imag < len(game_map[0])
                and pos.real < len(game_map),
                next_positions,
            )
        )
        next_positions = list(
            filter(
                lambda pos: game_map[int(pos.real)][int(pos.imag)] != "#"
                and (
                    pos != position - 1 or game_map[int(pos.real)][int(pos.imag)] != "v"
                )
                and (
                    pos != position + 1j
                    or game_map[int(pos.real)][int(pos.imag)] != "<"
                )
                and (
                    pos != position - 1j
                    or game_map[int(pos.real)][int(pos.imag)] != ">"
                ),
                next_positions,
            )
        )

        if len(next_positions) == 0:
            return None

        steps += 1

        if len(next_positions) == 1:
            position = next_positions[0]

        else:
            sub_steps = list(
                map(
                    lambda x: iterate(
                        input_string,
                        start_position=next_positions[x],
                        end_position=end_position,
                        visited=visited.copy(),
                    ),
                    range(len(next_positions)),
                )
            )
            sub_steps = list(filter(lambda x: x is not None, sub_steps))

            if len(sub_steps) == 0:
                return None
            else:
                return steps + max(sub_steps)

    return steps

In [173]:
test_input = """\
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#"""

In [174]:
assert iterate(test_input, 0 + 1j, get_end_position(test_input), set()) == 94

In [175]:
input = open("23.txt").read()
value = iterate(input, 0 + 1j, get_end_position(input), set())

print(f"Part 1: {value}")
assert value == 2386

Part 1: 2386


In [176]:
import re

test_input = re.sub(SLOPE_REGEX, ".", test_input)
assert iterate(test_input, 0 + 1j, get_end_position(test_input), set()) == 154

In [177]:
input = open("23.txt").read()
input = re.sub(SLOPE_REGEX, ".", input)
value = iterate(input, 0 + 1j, get_end_position(input), set())

print(f"Part 2: {value}")
# assert value == 2386